In [56]:
from pygments import highlight
from pygments.styles import get_style_by_name
from pygments.lexers import JavaLexer
from pygments.formatters import BBCodeFormatter
from PIL import Image, ImageDraw, ImageFont
import os
import re
import json

In [52]:
def parse_bbcode_list(bbcode):
    result = []
    pattern = '\[.*?\]'
    idx = 0
    base_color = (0,0,0,255)
    color = base_color
    bold = False
    italic = False
    aoi_default = "None"
    aoi_name = aoi_default
    while idx < len(bbcode):
        data = re.search(pattern, bbcode[idx:])
        meta = False
        
        if data is not None and data.start(0) == 0:
            idx += data.end(0)
            value = data.group(0).strip("[").strip("]")
            if '/AOI' in value:
                meta = True
                aoi_name = aoi_default
            elif 'AOI' in value:
                meta = True
                aoi_name = value[4:]
            
            if '/color' in value:
                meta = True
                color = base_color
            elif 'color' in value:
                meta = True
                color = tuple(int(value[i:i+2], 16) for i in (7, 9, 11))
                color = (color[0], color[1], color[2], 255)
                
            if '/b' in value:
                meta = True
                bold = False
            elif 'b' in value:
                meta = True
                bold = True
                
            if '/i' in value:
                meta = True
                italic = False
            elif 'i' in value:
                meta = True
                italic = True
        if meta == False:
            result.append({'letter':bbcode[idx],
                           'color':color,
                           'bold':bold,
                           'italic':italic,
                           'AOI': aoi_name})
            idx+=1
            
    return result

def set_font(data, regular, bold, italic, bolditalic):
    new_result = []
    for entry in data:
        if entry["bold"] == True and entry["italic"] == True:
            entry["font"] = bolditalic
        
        if entry["bold"] == True and entry["italic"] == False:
            entry["font"] = bold
        
        if entry["bold"] == False and entry["italic"] == True:
            entry["font"] = italic
        
        if entry["bold"] == False and entry["italic"] == False:
            entry["font"] = regular
            
        new_result.append(entry)
            
    return new_result

In [62]:
def create_image(source_json, font_path = "\\fonts\\ttf\\", lexer = JavaLexer):
    data = {}
    with open(source_json) as f:
        data = json.load(f)
    
    code = ""
    for line in data["source-code"]:
        code += line + "\n"

    
    width_margin = data["width-margin"]
    height_margin = data["height-margin"]
    spacing = data["spacing"]

    regular = ImageFont.truetype(os.getcwd() + font_path + data["font-normal"], data["font-size"], encoding="unic")
    bold = ImageFont.truetype(os.getcwd() + font_path + data["font-bold"], data["font-size"], encoding="unic")
    italic = ImageFont.truetype(os.getcwd() + font_path + data["font-italic"], data["font-size"], encoding="unic")
    bolditalic = ImageFont.truetype(os.getcwd() + font_path + data["font-bold-italic"], data["font-size"], encoding="unic")


    code = highlight(code, lexer(), BBCodeFormatter(line_numbers=False, style=get_style_by_name("vs")))
    code = parse_bbcode_list(code)
    code = set_font(code, regular, bold, italic, bolditalic)
    
    source_code = []
    tmp_source_code = ""
    for tmp_data in code:
        if tmp_data["letter"] == "\n":
            source_code.append(tmp_source_code)
            tmp_source_code = ""
        else:
            tmp_source_code += tmp_data["letter"]
    
    img = Image.new("RGBA", (1, 1), color = 'white')
    draw = ImageDraw.Draw(img)
    max_length = max([draw.textsize(txt, regular)[0] for txt in source_code]) + 2*width_margin
    max_height = sum([draw.textsize("placeholder", regular)[1]+spacing for txt in data["source-code"]]) + 2*height_margin

    img = Image.new("RGBA", (max_length, max_height), color = 'white')
    draw = ImageDraw.Draw(img)
    
    width_current = width_margin
    height_current = height_margin
    result = []
    for letter in code:
        if letter["letter"] == "\n":
            width_current = width_margin
            height_current = height_current + draw.textsize("placeholder", regular)[1] + spacing
            letter["BoundingBox"] = (0,0,0,0)
        else:
            letter["BoundingBox"] = draw.textbbox((width_current, height_current), letter["letter"], font=letter["font"])
            draw.text((width_current, height_current), letter["letter"], letter["color"], font=letter["font"])
            width_current += draw.textsize(letter["letter"], regular)[0]
            
        result.append(letter)       
    return img, result

In [64]:
def cluster_aois(data):
    distinct_aois = list(set([letter["AOI"] for letter in data]))
    del distinct_aois[distinct_aois.index("None")]
    aoi_dict = {aoi_name: [None,None,None,None] for aoi_name in distinct_aois}
    for letter in data: 
        for aoi_name in distinct_aois: 
            if letter["AOI"] == aoi_name:
                if aoi_dict[aoi_name][0] == None:
                    aoi_dict[aoi_name][0] = letter["BoundingBox"][0]
                    aoi_dict[aoi_name][1] = letter["BoundingBox"][1]
                    aoi_dict[aoi_name][2] = letter["BoundingBox"][2]
                    aoi_dict[aoi_name][3] = letter["BoundingBox"][3]
            
                if aoi_dict[aoi_name][0] > letter["BoundingBox"][0]:
                    aoi_dict[aoi_name][0] = letter["BoundingBox"][0]
            
                if aoi_dict[aoi_name][1] > letter["BoundingBox"][1]:
                    aoi_dict[aoi_name][1] = letter["BoundingBox"][1]
            
                if aoi_dict[aoi_name][2] < letter["BoundingBox"][2]:
                    aoi_dict[aoi_name][2] = letter["BoundingBox"][2]
            
                if aoi_dict[aoi_name][3] < letter["BoundingBox"][3]:
                    aoi_dict[aoi_name][3] = letter["BoundingBox"][3]
                    
    return aoi_dict

#img, result = create_image("ackermann.json")
#im = img.copy()
#draw = ImageDraw.Draw(im)
#
#for key, value in cluster_aois(result).items():
#    draw.rectangle(value, outline=(255, 0, 0))
#    
#im
#im.save("text.png", "PNG")